In [ ]:
import pandas as pd

def forward_fill(df, var_range, id_var):
    """
    Forward fills missing values in a specified range of variables based on the last known value for each ID.

    Parameters:
    - df (pd.DataFrame): The dataset.
    - var_range (list): List of variables to forward fill (e.g., ["weight", "height"]).
    - id_var (str): The name of the ID variable (e.g., "id").
    
    Returns:
    - pd.DataFrame: DataFrame with forward-filled values.
    """
    df = df.sort_values(by=[id_var]).reset_index(drop=True)  # Ensure proper sorting
    for var in var_range:
        df[var] = df.groupby(id_var)[var].ffill()
    return df


def sort_if_needed(df, id_var, date_var):
    """
    Checks if the dataset is already sorted by `id_var` and `date_var`, and sorts if necessary.

    Parameters:
    - df (pd.DataFrame): The dataset.
    - id_var (str): The name of the ID variable (e.g., "id").
    - date_var (str): The name of the date variable (e.g., "date").

    Returns:
    - pd.DataFrame: Sorted DataFrame.
    """
    # Check if sorted
    is_sorted = (df[id_var].is_monotonic_increasing and
                 df.groupby(id_var)[date_var].is_monotonic_increasing.all())
    
    if not is_sorted:
        df = df.sort_values(by=[id_var, date_var]).reset_index(drop=True)
    
    return df


def back_and_forward_fill_with_date(df, var_range, id_var, date_var):
    """
    Back and forward fills missing values in a specified range of variables based on the last known value for each ID and date.

    Parameters:
    - df (pd.DataFrame): The dataset.
    - var_range (list): List of variables to back and forward fill (e.g., ["weight", "height"]).
    - id_var (str): The name of the ID variable (e.g., "id").
    - date_var (str): The name of the date variable (e.g., "date").

    Returns:
    - pd.DataFrame: DataFrame with back and forward-filled values.
    """
    # Ensure sorting
    df = sort_if_needed(df, id_var, date_var)
    
    for var in var_range:
        # Back fill
        df[var] = df.groupby(id_var)[var].bfill()
        # Forward fill
        df[var] = df.groupby(id_var)[var].ffill()

    return df


In [ ]:
# Example dataset
data = {
    "id": [1, 1, 1, 2, 2, 2],
    "date": ["2023-01-01", "2023-01-02", "2023-01-03", "2023-01-01", "2023-01-02", "2023-01-03"],
    "weight": [70, None, 72, None, 80, None],
    "height": [160, None, None, 170, None, 175]
}
df = pd.DataFrame(data)
df["date"] = pd.to_datetime(df["date"])

# Forward fill
df = forward_fill(df, var_range=["weight", "height"], id_var="id")
print("Forward Fill:\n", df)

# Sort if needed
df = sort_if_needed(df, id_var="id", date_var="date")
print("Sorted Data:\n", df)

# Back and forward fill with date
df = back_and_forward_fill_with_date(df, var_range=["weight", "height"], id_var="id", date_var="date")
print("Back and Forward Fill:\n", df)
